### Kaggle challenge:

https://www.kaggle.com/c/whats-cooking-kernels-only

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import time

In [2]:
np.random.seed(1988)

In [3]:
! dir

 El volumen de la unidad C no tiene etiqueta.
 El n£mero de serie del volumen es: 48E8-1CA4

 Directorio de C:\Users\SynergicTemp\Downloads\4 - Cuisine_categorization

02/07/2018  15:21    <DIR>          .
02/07/2018  15:21    <DIR>          ..
02/07/2018  10:17    <DIR>          .ipynb_checkpoints
26/06/2018  15:53         1.540.094 array.csv
27/06/2018  17:28           113.531 Cuisine.ipynb
02/07/2018  15:21           190.706 Kaggle_L”sung.ipynb
25/06/2018  17:23            26.415 sample_submission.csv.zip
13/05/2018  18:21         2.844.086 test.json
13/05/2018  18:21        12.415.067 train.json
               6 archivos     17.129.899 bytes
               3 dirs  397.073.944.576 bytes libres


### Data exploration

In [4]:
data = pd.read_json("./train.json")
data.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [5]:
#there is no NaNs
data[pd.isna(data['ingredients'])]

,cuisine,id,ingredients


In [6]:
data.shape

(39774, 3)

In [7]:
def extract_ingredients(serie):
    list_ingredients=[]
    for lista in serie:
        for element in lista:
            if element in list_ingredients:
                pass
            elif element not in list_ingredients:
                list_ingredients.append(element)
            else:
                pass
        
    return list_ingredients      

In [8]:
ingredients = extract_ingredients(data['ingredients'])

In [9]:
len (ingredients)

6714

In [10]:
#Types of differents cuisines:
data['cuisine'].unique().shape

(20,)

In [11]:
cuisines = data['cuisine'].unique()
cuisines

array(['greek', 'southern_us', 'filipino', 'indian', 'jamaican',
       'spanish', 'italian', 'mexican', 'chinese', 'british', 'thai',
       'vietnamese', 'cajun_creole', 'brazilian', 'french', 'japanese',
       'irish', 'korean', 'moroccan', 'russian'], dtype=object)

We will try two feature engineering methods, one hot encoding and feature hashing to see what is more effective.

## One hot encoding method

In [12]:
#Create columns
t = time.time()
for ingredient in ingredients:
    data[ingredient]=np.zeros(len(data["ingredients"]))

print("It took %i seg" %(time.time()-t))

It took 129 seg


In [13]:
def ohe(serie):    
    ind=0
    for lista in serie:
        for ingredient in lista:
            data.loc[ind,ingredient]=1
        ind +=1

In [14]:
t = time.time()
ohe(data['ingredients'])
print('it took %i segs' % (time.time()-t))

it took 228 segs


#### Train / test split

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
predictors = ingredients
response = 'cuisine'

In [17]:
data.head()

,cuisine,id,ingredients,romaine lettuce,black olives,grape tomatoes,garlic,pepper,purple onion,seasoning,...,Oscar Mayer Cotto Salami,Challenge Butter,orange glaze,cholesterol free egg substitute,ciabatta loaf,LiptonÂ® Iced Tea Brew Family Size Tea Bags,Hidden ValleyÂ® Greek Yogurt Original RanchÂ® Dip Mix,lop chong,tomato garlic pasta sauce,crushed cheese crackers
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...",1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,indian,22213,"[water, vegetable oil, wheat, salt]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
X = data[predictors]
Y = data[response]

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)

In [20]:
del(data)

#### Decision tree classifier

In [20]:
from sklearn.tree import DecisionTreeClassifier

In [21]:
tree_model = DecisionTreeClassifier(criterion='gini', max_depth=50, min_samples_leaf=10)

In [22]:
tree_model.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=50,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [23]:
output_tree = tree_model.predict(x_test)

In [24]:
output_tree

array(['italian', 'italian', 'chinese', ..., 'mexican', 'mexican',
       'italian'], dtype=object)

In [25]:
from sklearn.metrics import *

In [26]:
accuracy_score(y_test,output_tree)

0.5518541797611565

In [28]:
conf_matr= pd.DataFrame(confusion_matrix(y_test,output_tree, labels=cuisines), index=cuisines, columns=cuisines)

In [29]:
conf_matr

,greek,southern_us,filipino,indian,jamaican,spanish,italian,mexican,chinese,british,thai,vietnamese,cajun_creole,brazilian,french,japanese,irish,korean,moroccan,russian
greek,104,55,0,3,0,6,46,11,2,3,0,0,0,0,12,0,1,0,5,0
southern_us,2,638,0,4,2,1,64,30,9,7,1,0,33,0,35,5,7,1,4,2
filipino,2,54,42,2,0,0,3,2,24,1,9,3,0,0,3,1,0,3,0,0
indian,1,92,0,409,6,2,12,48,5,0,5,0,5,0,6,6,0,2,16,0
jamaican,0,46,3,12,13,0,5,8,4,0,4,1,2,0,4,0,0,0,1,0
spanish,1,46,0,2,0,33,63,22,0,0,0,0,8,0,24,4,1,0,6,0
italian,31,326,1,9,0,9,1085,20,3,2,0,1,13,0,74,2,0,0,3,3
mexican,4,195,3,22,1,0,54,963,5,2,4,1,19,1,13,4,0,0,12,0
chinese,2,70,13,2,0,0,23,7,361,0,13,5,0,0,0,37,0,11,0,0
british,0,102,0,5,0,2,16,0,3,7,0,1,2,0,15,0,1,0,0,3


In [30]:
print(classification_report(y_test,output_tree))

              precision    recall  f1-score   support

   brazilian       0.83      0.15      0.25       101
     british       0.23      0.04      0.07       157
cajun_creole       0.57      0.45      0.50       305
     chinese       0.65      0.66      0.66       544
    filipino       0.47      0.28      0.35       149
      french       0.38      0.28      0.32       501
       greek       0.66      0.42      0.51       248
      indian       0.77      0.67      0.71       615
       irish       0.46      0.10      0.17       115
     italian       0.69      0.69      0.69      1582
    jamaican       0.50      0.13      0.20       103
    japanese       0.60      0.47      0.53       272
      korean       0.64      0.34      0.44       170
     mexican       0.80      0.74      0.77      1303
    moroccan       0.53      0.41      0.46       156
     russian       0.21      0.03      0.05       103
 southern_us       0.28      0.76      0.41       845
     spanish       0.46    

In [31]:
#Reals
italia = 0
for i in y_test:
    if i=='italian':
        italia+=1

italia

1582

In [32]:
#Predicted
italia = 0
for i in output_tree:
    if i=='italian':
        italia+=1

italia

1573

#### Log regression:

In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
log_reg = LogisticRegression(C=1)

In [29]:
log_reg.fit(x_train, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [30]:
y_predicted = log_reg.predict(x_test)
y_predicted

array(['italian', 'italian', 'chinese', ..., 'mexican', 'mexican',
       'italian'], dtype=object)

In [31]:
accuracy_score(y_test, y_predicted)

0.7786297925832809

In [39]:
pd.DataFrame(confusion_matrix(y_test, y_predicted, labels=cuisines), index=cuisines, columns=cuisines)

,greek,southern_us,filipino,indian,jamaican,spanish,italian,mexican,chinese,british,thai,vietnamese,cajun_creole,brazilian,french,japanese,irish,korean,moroccan,russian
greek,155,4,0,9,0,6,46,4,0,0,0,0,0,0,13,0,2,1,6,2
southern_us,0,685,1,2,2,4,44,25,6,2,1,0,34,0,35,1,2,0,0,1
filipino,0,11,93,2,0,1,3,6,19,0,3,5,1,0,1,2,1,0,0,1
indian,2,4,3,546,2,0,11,17,2,1,4,1,0,0,2,7,0,2,10,1
jamaican,0,11,1,2,67,1,2,5,0,1,2,0,3,2,3,0,1,0,1,1
spanish,4,8,3,4,0,92,42,18,0,2,0,0,6,0,28,0,0,0,3,0
italian,15,28,0,8,1,5,1421,17,1,4,1,0,8,0,58,2,4,0,3,6
mexican,2,34,4,4,1,3,36,1196,3,0,1,0,3,5,7,1,0,0,3,0
chinese,0,5,6,2,0,0,12,5,469,0,13,5,2,0,2,14,0,9,0,0
british,2,39,1,2,3,1,15,2,1,50,0,0,1,0,24,0,13,0,1,2


#### Random forest:

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
RF = RandomForestClassifier(n_estimators=10, max_depth=20, min_samples_leaf=20)

In [22]:
RF.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=20, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [23]:
y_predicted = RF.predict(x_test)
y_predicted

array(['italian', 'italian', 'chinese', ..., 'mexican', 'mexican',
       'italian'], dtype=object)

In [25]:
accuracy_score(y_test, y_predicted)

0.4609679446888749

In [66]:
#free memory
% reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


#### Adaboost + logistic regression

In [26]:
from sklearn.ensemble import AdaBoostClassifier

In [47]:
DT =  DecisionTreeClassifier(max_depth=20, min_samples_leaf=30)

In [48]:
AB_Class = AdaBoostClassifier(base_estimator=DT, n_estimators=50, learning_rate=1)

In [50]:
t=time.time()
AB_Class.fit(x_train,y_train)
print("This took %i seconds" %(time.time()-t))

This took 882 seconds


In [51]:
y_predicted = AB_Class.predict(x_test)
y_predicted

array(['spanish', 'italian', 'chinese', ..., 'mexican', 'mexican',
       'italian'], dtype=object)

In [52]:
accuracy_score(y_test, y_predicted)

0.5214330609679447

## Feature Hashing method

In [102]:
data.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [76]:
from sklearn.feature_extraction import FeatureHasher

In [77]:
hasher = FeatureHasher(n_features=20, input_type='string', non_negative=True)

C:\Program Files\Anaconda\lib\site-packages\sklearn\feature_extraction\hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [78]:
data['ingredients'][0]

['romaine lettuce',
 'black olives',
 'grape tomatoes',
 'garlic',
 'pepper',
 'purple onion',
 'seasoning',
 'garbanzo beans',
 'feta cheese crumbles']

In [79]:
a= hasher.transform(data['ingredients'])

In [80]:
a.toarray()

array([[0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 1., ..., 1., 0., 1.],
       [0., 1., 1., ..., 1., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 1., ..., 1., 1., 2.],
       [0., 0., 1., ..., 1., 2., 0.]])

In [81]:
hashed = pd.DataFrame(a.toarray())
hashed.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0
2,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0,0.0,1.0,1.0,2.0,1.0


In [82]:
hashed.shape

(39774, 20)

In [83]:
data.shape

(39774, 3)

In [85]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(hashed, data['cuisine'], test_size=0.2)

#### Decision tree classifier

In [86]:
from sklearn.tree import DecisionTreeClassifier

In [94]:
tree_hashed = DecisionTreeClassifier(criterion='gini', max_depth=20, min_samples_leaf=10)

In [95]:
tree_hashed.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=20,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [96]:
y_predicted = tree_hashed.predict(x_test)
y_predicted

array(['french', 'italian', 'indian', ..., 'italian', 'mexican',
       'mexican'], dtype=object)

In [97]:
from sklearn.metrics import *

In [98]:
accuracy_score(y_test, y_predicted)

0.21671904462602137

In [99]:
data['cuisine'].value_counts()

italian         7838
mexican         6438
southern_us     4320
indian          3003
chinese         2673
french          2646
cajun_creole    1546
thai            1539
japanese        1423
greek           1175
spanish          989
korean           830
vietnamese       825
moroccan         821
british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        467
Name: cuisine, dtype: int64

In [100]:
pd.DataFrame(confusion_matrix(y_test, y_predicted, labels=cuisines), index=cuisines, columns=cuisines)

,greek,southern_us,filipino,indian,jamaican,spanish,italian,mexican,chinese,british,thai,vietnamese,cajun_creole,brazilian,french,japanese,irish,korean,moroccan,russian
greek,10,26,1,21,1,1,70,53,16,1,2,0,6,1,29,5,3,2,0,0
southern_us,6,158,5,52,3,2,304,126,52,6,8,3,40,5,48,11,7,5,4,0
filipino,1,11,0,11,0,0,45,28,26,2,6,1,5,1,7,4,1,0,0,0
indian,10,40,7,161,3,2,142,122,41,4,10,1,26,1,29,6,2,0,6,2
jamaican,1,19,0,10,1,1,23,20,8,1,7,0,8,1,2,0,0,0,1,0
spanish,3,19,0,17,0,1,82,31,17,3,2,0,11,2,17,2,1,0,2,0
italian,34,203,9,82,1,2,669,259,80,12,13,4,67,17,92,18,9,4,7,0
mexican,14,95,9,108,5,3,319,488,79,10,23,5,51,2,57,18,0,5,11,1
chinese,2,42,9,45,1,1,117,112,121,6,9,1,25,3,34,14,2,0,0,0
british,6,24,2,8,0,1,55,23,8,5,0,1,1,1,18,1,1,1,1,0


#### Logistic regression:

In [101]:
from sklearn.linear_model import LogisticRegression

In [105]:
lr = LogisticRegression()

In [106]:
lr.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [107]:
y_predicted = lr.predict(x_test)

In [108]:
accuracy_score(y_test,y_predicted)

0.28296668761785043

In [110]:
print(classification_report(y_test,y_predicted))

              precision    recall  f1-score   support

   brazilian       0.00      0.00      0.00       101
     british       0.00      0.00      0.00       157
cajun_creole       0.21      0.03      0.05       305
     chinese       0.28      0.29      0.29       544
    filipino       0.00      0.00      0.00       149
      french       0.18      0.00      0.01       501
       greek       0.33      0.01      0.02       248
      indian       0.30      0.27      0.28       615
       irish       0.00      0.00      0.00       115
     italian       0.28      0.67      0.39      1582
    jamaican       0.00      0.00      0.00       103
    japanese       0.20      0.00      0.01       272
      korean       0.00      0.00      0.00       170
     mexican       0.32      0.55      0.40      1303
    moroccan       0.29      0.01      0.02       156
     russian       0.00      0.00      0.00       103
 southern_us       0.20      0.15      0.17       845
     spanish       0.00    

C:\Program Files\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
